In [2]:
import pandas as pd
import numpy as np
import sklearn.metrics as metrics

from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, RidgeCV
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

from sklearn.model_selection import train_test_split

data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
answers = pd.read_csv("sample_submission.csv")
data.head()

,Index,Gender,Senior,Married,Dependents,Tenure,Phone,MultiplePhones,Internet,Security,...,Insurance,Support,TV,Movies,Contract,EBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,Female,0,Yes,Yes,67,No,No phone service,DSL,Yes,...,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),54.20,3623.95,False
1,1,Female,0,No,No,34,Yes,Yes,DSL,Yes,...,No,Yes,No,No,One year,Yes,Mailed check,64.40,2088.75,True
2,2,Female,1,Yes,No,72,Yes,Yes,Fiber optic,Yes,...,Yes,Yes,No,No,Two year,Yes,Bank transfer (automatic),96.60,6827.5,False
3,3,Female,0,Yes,Yes,63,Yes,No,DSL,Yes,...,Yes,Yes,No,No,Two year,No,Mailed check,63.55,4014.2,False
4,4,Male,0,Yes,Yes,69,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,No,One year,No,Credit card (automatic),95.75,6511.25,False


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3591 entries, 0 to 3590
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Index           3591 non-null   int64  
 1   Gender          3591 non-null   object 
 2   Senior          3591 non-null   int64  
 3   Married         3591 non-null   object 
 4   Dependents      3591 non-null   object 
 5   Tenure          3591 non-null   int64  
 6   Phone           3591 non-null   object 
 7   MultiplePhones  3591 non-null   object 
 8   Internet        3591 non-null   object 
 9   Security        3591 non-null   object 
 10  Backup          3591 non-null   object 
 11  Insurance       3591 non-null   object 
 12  Support         3591 non-null   object 
 13  TV              3591 non-null   object 
 14  Movies          3591 non-null   object 
 15  Contract        3591 non-null   object 
 16  EBilling        3591 non-null   object 
 17  PaymentMethod   3591 non-null   o

In [4]:
data.describe()

,Index,Senior,Tenure,MonthlyCharges
count,3591.000000,3591.000000,3591.000000,3591.000000
mean,1795.000000,0.156502,31.971039,64.487650
std,1036.776736,0.363382,24.579624,30.057029
min,0.000000,0.000000,0.000000,18.400000
25%,897.500000,0.000000,8.000000,35.400000
50%,1795.000000,0.000000,28.000000,70.300000
75%,2692.500000,0.000000,55.000000,89.450000
max,3590.000000,1.000000,72.000000,118.750000


In [5]:
gender = {'Female': 0, 'Male': 1}
yesNo = {'No': 0, 'Yes': 1}
multPhones = {'No': 0, 'Yes': 1, 'No phone service': -1}
internet = {'DSL': 1, 'Fiber optic': 2, 'No': 0}
yesNoInternet = {'No': 0, 'Yes': 1, 'No internet service': -1}
contract = {'Month-to-month': 0, 'One year': 1, 'Two year': 2}
paymentMethod = {'Electronic check' : 0, 'Mailed check' : 1, 'Bank transfer (automatic)' : 2, 'Credit card (automatic)': 3}
cleanData = data.replace({'Gender': gender, 'Married': yesNo, 'Dependents': yesNo, 'Phone': yesNo, 
                   'MultiplePhones': multPhones, 'Internet': internet, 'Security': yesNoInternet, 'Backup': yesNoInternet, 
                   'Insurance': yesNoInternet, 'Support': yesNoInternet, 'TV': yesNoInternet, 'Movies': yesNoInternet,
                   'Contract': contract, 'EBilling': yesNo, 'PaymentMethod': paymentMethod})
cleanData.TotalCharges = pd.to_numeric(cleanData.TotalCharges,errors = 'coerce')
cleanData.TotalCharges = cleanData.TotalCharges.fillna(0)
cleanData.drop(columns=['Index'], inplace = True)
# cleanData.replace('Married', yesNo)
# cleanData.replace('Dependents', yesNo)
# cleanData.Gender = (data.Gender == 'Male')
# cleanData.Gender = [hash(item) for item in data.Gender]
# cleanData.Gender = cleanData.Gender - cleanData.Gender.min()
# cleanData.Gender = [1 - item // cleanData.Gender.max() for item in data.Gender]
# cleanData.Married = data.Married = 'Yes'
cleanData

,Gender,Senior,Married,Dependents,Tenure,Phone,MultiplePhones,Internet,Security,Backup,Insurance,Support,TV,Movies,Contract,EBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,0,1,1,67,0,-1,1,1,1,1,1,1,0,2,0,3,54.20,3623.95,False
1,0,0,0,0,34,1,1,1,1,1,0,1,0,0,1,1,1,64.40,2088.75,True
2,0,1,1,0,72,1,1,2,1,1,1,1,0,0,2,1,2,96.60,6827.50,False
3,0,0,1,1,63,1,0,1,1,1,1,1,0,0,2,0,1,63.55,4014.20,False
4,1,0,1,1,69,1,1,2,0,1,1,0,1,0,1,0,3,95.75,6511.25,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3586,0,0,0,0,5,1,0,2,0,0,0,0,1,0,0,0,0,80.20,384.25,False
3587,0,0,1,1,71,1,0,1,1,0,1,0,1,1,2,0,3,77.55,5574.35,False
3588,0,0,1,0,68,1,1,2,0,1,1,0,0,0,1,0,3,86.45,5762.95,False
3589,0,0,1,1,12,1,0,1,0,0,1,0,1,0,0,1,3,58.35,740.55,False


In [6]:
test.head()

,Index,Gender,Senior,Married,Dependents,Tenure,Phone,MultiplePhones,Internet,Security,Backup,Insurance,Support,TV,Movies,Contract,EBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,0,Male,0,No,No,58,Yes,No,Fiber optic,Yes,Yes,No,No,Yes,Yes,One year,No,Bank transfer (automatic),99.15,5720.95
1,1,Female,0,Yes,Yes,26,Yes,No,Fiber optic,No,No,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,90.80,2361.8
2,2,Female,0,No,No,26,Yes,No,Fiber optic,Yes,No,Yes,Yes,No,No,One year,Yes,Credit card (automatic),87.15,2274.1
3,3,Male,0,No,No,12,Yes,Yes,DSL,Yes,Yes,No,Yes,No,No,Month-to-month,Yes,Electronic check,67.25,832.3
4,4,Female,0,No,No,1,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,75.05,75.05


In [7]:
cleanTest = test.replace({'Gender': gender, 'Married': yesNo, 'Dependents': yesNo, 'Phone': yesNo, 
                   'MultiplePhones': multPhones, 'Internet': internet, 'Security': yesNoInternet, 'Backup': yesNoInternet, 
                   'Insurance': yesNoInternet, 'Support': yesNoInternet, 'TV': yesNoInternet, 'Movies': yesNoInternet,
                   'Contract': contract, 'EBilling': yesNo, 'PaymentMethod': paymentMethod})
cleanTest.TotalCharges = pd.to_numeric(cleanTest.TotalCharges,errors = 'coerce')
cleanTest.TotalCharges = cleanTest.TotalCharges.fillna(0)
cleanTest.drop(columns=['Index'], inplace = True)
cleanTest

,Gender,Senior,Married,Dependents,Tenure,Phone,MultiplePhones,Internet,Security,Backup,Insurance,Support,TV,Movies,Contract,EBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,1,0,0,0,58,1,0,2,1,1,0,0,1,1,1,0,2,99.15,5720.95
1,0,0,1,1,26,1,0,2,0,0,0,0,1,1,0,1,0,90.80,2361.80
2,0,0,0,0,26,1,0,2,1,0,1,1,0,0,1,1,3,87.15,2274.10
3,1,0,0,0,12,1,1,1,1,1,0,1,0,0,0,1,0,67.25,832.30
4,0,0,0,0,1,1,1,2,0,0,0,0,0,0,0,1,1,75.05,75.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2390,1,0,0,0,62,0,-1,1,1,1,1,1,0,0,2,1,2,43.70,2618.30
2391,0,0,1,0,2,1,0,0,-1,-1,-1,-1,-1,-1,0,0,2,20.10,43.15
2392,0,0,1,1,23,1,1,2,0,0,0,0,1,0,0,1,0,83.75,1849.95
2393,1,1,1,0,53,1,1,2,0,1,0,0,1,1,0,1,0,101.90,5549.40


In [45]:
X, Y = cleanData.drop(columns=['Churn']), cleanData['Churn']
# X_train, Y_train = X, Y
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=10)

In [50]:
X_train.info()
Y_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2513 entries, 1906 to 1289
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Gender          2513 non-null   int64  
 1   Senior          2513 non-null   int64  
 2   Married         2513 non-null   int64  
 3   Dependents      2513 non-null   int64  
 4   Tenure          2513 non-null   int64  
 5   Phone           2513 non-null   int64  
 6   MultiplePhones  2513 non-null   int64  
 7   Internet        2513 non-null   int64  
 8   Security        2513 non-null   int64  
 9   Backup          2513 non-null   int64  
 10  Insurance       2513 non-null   int64  
 11  Support         2513 non-null   int64  
 12  TV              2513 non-null   int64  
 13  Movies          2513 non-null   int64  
 14  Contract        2513 non-null   int64  
 15  EBilling        2513 non-null   int64  
 16  PaymentMethod   2513 non-null   int64  
 17  MonthlyCharges  2513 non-null 

AttributeError: AttributeError: 'Series' object has no attribute 'info'

In [56]:
std_scaler = StandardScaler()

X_train_scaled = std_scaler.fit_transform(X_train)
X_test_scaled = std_scaler.transform(X_test)
# test_scaled = std_scaler.transform(cleanTest)

X_train_scaled

array([[ 0.98931297, -0.43120385, -0.97136098, ...,  1.44956839,
         1.14848925,  1.70620123],
       [-1.01080247, -0.43120385, -0.97136098, ..., -0.28425615,
        -1.44807883, -0.96683219],
       [-1.01080247, -0.43120385,  1.0294834 , ...,  0.58265612,
        -1.47630239, -0.53547443],
       ...,
       [ 0.98931297, -0.43120385, -0.97136098, ..., -0.28425615,
        -1.4945647 , -0.46824842],
       [-1.01080247,  2.31908874,  1.0294834 , ..., -1.15116843,
         1.18335366,  2.00625989],
       [ 0.98931297,  2.31908874,  1.0294834 , ..., -1.15116843,
         0.78490331,  0.81907798]])

In [60]:
# len(Y_test)
len(predictions)

1078

In [65]:
model = LassoCV()

model.fit(X_train_scaled, Y_train)
predictions = model.predict(X_test_scaled)

res = [round(item) for item in predictions]
# d = {'Index': cleanTest.index, 'Churn': res}
# df = pd.DataFrame(data=d)
# df.Churn = df.Churn == 1
# df
# true = answers.Churn * 1
# pred = df.Churn * 1
res

# true
# len(Y_test)
# len(predictions)
metrics.matthews_corrcoef(Y_test * 1, res)

# df.set_index('Index')
# df
# df.to_csv("answers.csv", index=False)
# metrics.matthews_corrcoef(true, pred)

0.44636198785947934

In [37]:
import math
def mcc(ytrue, ypred):
    tp = 0
    tn = 0
    fp = 0
    fn = 0

    for i in range(0, len(ytrue)):
        if ytrue[i] and ypred[i]:
            tp += 1
        if not ytrue[i] and not ypred[i]:
            tn += 1
        if ytrue[i] and not ypred[i]:
            fn += 1
        if not ytrue[i] and ypred[i]:
            fp += 1
    

    tpfp = tp + fp
    tpfn = tp + fn
    tnfp = tn + fp
    tnfn = tn + fn

    if tpfp == 0:
        tpfp = 1
    if tpfn == 0:
        tpfn = 1
    if tnfp == 0:
        tnfp = 1
    if tnfn == 0:
        tnfn = 1
    return (tp * tn - fp * fn) / math.sqrt(tpfp * tpfn * tnfp * tnfn)

actual = [1,1,1,1,1,1,1,1,0,0,0,0]
prediction = [0,0,1,1,1,1,1,1,0,0,0,1]

mcc(answers.Churn.to_list() + [True, True, False, False], df.Churn.to_list() + [True, False, False, True])

0.022716772538094548

In [35]:
type(answers.Churn.array)

pandas.core.arrays.numpy_.PandasArray